## Imports

In [ ]:
from keras.api.callbacks import EarlyStopping
from keras.api.datasets import imdb
from keras.api.layers import Dense, Embedding, Input, SimpleRNN
from keras.api.models import Sequential
from keras.api.preprocessing import sequence

In [2]:
%env TF_ENABLE_ONEDNN_OPTS=1

env: TF_ENABLE_ONEDNN_OPTS=1


## Load the IMDB dataset

In [3]:
max_features = 10000 # vocab size
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

print(f"Traning data shape: {X_train.shape}, Testing data shape: {X_test.shape}")
print(f"Traning label shape: {y_train.shape}, Testing label shape: {y_test.shape}")

Traning data shape: (25000,), Testing data shape: (25000,)
Traning label shape: (25000,), Testing label shape: (25000,)


In [4]:
# Inspect a sample review and it's label
print(f"Review: {X_train[0]}")
print(f"Label: {y_train[0]}")

Review: [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
Label: 1


In [5]:
# Decode the review back to English
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in X_train[0]])
print(f"Decoded review: {decoded_review}")

Decoded review: ? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have do

## Pad the sequence

In [6]:
X_train = sequence.pad_sequences(X_train, maxlen=500)
X_test = sequence.pad_sequences(X_test, maxlen=500)

## Train the Simple RNN

In [10]:
model = Sequential([
    Input(shape=(500,)),
    Embedding(max_features, 128),
    SimpleRNN(128, activation="relu"),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,025 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# Create callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
]

In [13]:
history = model.fit(
    X_train,
    y_train,
    epochs=10,
    batch_size=128,
    validation_split=0.2,
    callbacks=callbacks,
)

Epoch 1/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 31s 188ms/step - accuracy: 0.5506 - loss: 605.9391 - val_accuracy: 0.5524 - val_loss: 0.7061
Epoch 2/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 43s 275ms/step - accuracy: 0.5606 - loss: 0.6784 - val_accuracy: 0.5858 - val_loss: 0.6539
Epoch 3/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 31s 194ms/step - accuracy: 0.6181 - loss: 493604.9375 - val_accuracy: 0.5062 - val_loss: 351325.5000
Epoch 4/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 30s 190ms/step - accuracy: 0.6028 - loss: 666506.3125 - val_accuracy: 0.6284 - val_loss: 0.6335
Epoch 5/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 29s 183ms/step - accuracy: 0.6861 - loss: 0.5906 - val_accuracy: 0.6302 - val_loss: 0.6303
Epoch 6/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 28s 180ms/step - accuracy: 0.6891 - loss: 0.5847 - val_accuracy: 0.6382 - val_loss: 0.6219
Epoch 7/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 31s 195ms/step - accuracy: 0.6991 - loss: 0.5637 - val_accuracy: 0.6514 - val_loss: 0.6119
Epoch 8/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 32s 200ms/step - accuracy:

In [15]:
# Save the model
model.save('../models/imdb_rnn.keras')